In [ ]:
# ИМПОРТ НЕОБХОДИМЫХ БИБЛИОТЕК

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st

In [ ]:
# ЧТЕНИЕ ФАЙЛА, ФОРМИРОВАНИЕ ДАТАСЕТА, ПЕРВИЧНАЯ ОЦЕНКА

credits = pd.read_csv('credit.csv', delimiter=';')
credits.head(100)

In [ ]:
#  КОЛИЧЕСТВО ЗАПИСЕЙ (17046. см. также credits.shape[0]), ТИПЫ ДАННЫХ, ЧИСЛО ЗАПОЛНЕННЫХ ЗАПИСЕЙ

credits.info()

In [ ]:
# ПРИВЕДЕНИЕ ТИПОВ ДАННЫХ: object -> float

credits['score_shk'] = credits['score_shk'].replace(',', '.',regex=True)
credits['score_shk'] = credits['score_shk'].astype('float64')

In [ ]:
credits['credit_sum'] = credits['credit_sum'].replace(',', '.',regex=True)
credits['credit_sum'] = credits['credit_sum'].astype('float64')

In [ ]:
# убеждаемся в том, что все типы приведены к numeric, что позволит вычислить статистики
credits.info()

In [ ]:
# ОСНОВНЫЕ СТАТИСТИКИ

credits.describe()

# после приведения типов располагаем статистиками всех признаков

In [ ]:
# ПРОВЕРКА КЛИЕНТОВ НА УНИКАЛЬНОСТЬ

if credits['client_id'].nunique() == credits.shape[0]:
    print('все клиенты уникальные')
else:
    print('число повторяющихся клиентов: ', credits.shape[0]-credits['client_id'].nunique())


In [ ]:
# ПРОВЕРКА НАБОРОВ ДАННЫХ НА УНИКАЛЬНОСТЬ

if credits.duplicated().sum() == 0:
    print('все записи уникальные по набору данных')
else:
    print(f'имеется {credits.duplicated().sum()} повторяющихся записей')


In [ ]:
# ПРОПУСКИ В ДАННЫХ

credits.isnull().sum()

<p>ЗАПОЛНЕНИЕ ПРОПУСКОВ</p>

<p>1. Пропуски в численных признаках credit_sum, montly_income, age, score_shk заместим медианными значениями</p>

In [ ]:
replaceables = ['monthly_income','age','credit_sum','score_shk']

for col in replaceables:
    medianValue = credits[col].median()
    credits[col] = credits[col].fillna(medianValue)
    print(col, ': ', credits[col].isnull().sum())

<p>2. Составляющие существенную долю пропуски credit_count и overdue_credit_count заместим медианными значениями по соответствующему региону</p>

In [ ]:
living_region_values = credits['living_region'].unique()

print("living_region_values: ", living_region_values)

for col in living_region_values:

    # 2 варианта - для сверки и учебы
    credit_count_median = credits.query('living_region == @col')['credit_count'].median()
    overdue_credit_count_median = credits.query('living_region == @col')['overdue_credit_count'].median()

    credit_count_median_2 = credits.loc[credits['living_region'] == col,'credit_count'].median()
    overdue_credit_count_median_2 = credits.loc[credits['living_region'] == col,'overdue_credit_count'].median()

    print('living_region =', col,
        ': credit_count_median = ', credit_count_median,'|', credit_count_median_2,
        ': overdue_credit_count_median =', overdue_credit_count_median, '|', overdue_credit_count_median_2)


    credits.loc[credits['living_region'] == col,'credit_count'] = credits.query('living_region == @col')['credit_count'].fillna(credit_count_median)
    credits.loc[credits['living_region'] == col,'overdue_credit_count'] = credits.query('living_region == @col')['overdue_credit_count'].fillna(overdue_credit_count_median)

print('credit_count: ', credits['credit_count'].isnull().sum())
print('overdue_credit_count: ', credits['overdue_credit_count'].isnull().sum())

ВЫВОДЫ:<br>
Пропуски по количеству кредитов и просроченных кредитов упали с 9230 до 18 - ненулевое количество объясняется тем, что есть одновременные пропуски и в living_region и в числе (просроченных) кредитов<br>
Однако оценить гипотезы по этим признакам для известных регионов теперь возможно

<p>3. Поскольку living_region, okrug, education и marital_status - категориальные признаки, а не скаляр, хоть и выражены числовым кодом.<br>
какое-либо определенное значение для замены пропуска в этих признаках не является оправданным.</p>
<p>Данные в этих записях являются полезными для сбора статистики по другим критериям, поэтому лучше бы оставить как есть<br>
однако поскольку хотелось бы избавиться на графиках от незначащих нулей в дробной части, а приведение float to int, оказывается, не работает, если в наборе имеется NaN,
то из-за незначительности процента пропуска записей с незаполненными регионами позволим себе 18 записей удалить</p>

In [ ]:
credits = credits.dropna()

In [ ]:
# ПРИВЕДЕНИЕ ТИПОВ ДАННЫХ: float -> int для удобства отображения
credits['credit_count'] = credits['credit_count'].astype('int64')
credits['overdue_credit_count'] = credits['overdue_credit_count'].astype('int64')
credits['education'] = credits['education'].astype('int64')
credits['living_region'] = credits['living_region'].astype('int64')
credits['marital_status'] = credits['marital_status'].astype('int64')
credits['okrug'] = credits['okrug'].astype('int64')
credits['age'] = credits['age'].astype('int64')


<p></p><br>
Сбор некоторой информации методами математической статистики

In [ ]:
# РАСПРЕДЕЛЕНИЕ ЗАЕМЩИКОВ ПО РЕГИОНАМ

credits['living_region'].value_counts()

In [ ]:
# В абсолютных значениях 
plt.subplot(1,1,1)
sns.countplot(data = credits, x = 'living_region', color = '#B234E1')

In [ ]:
# РАСПРЕДЕЛЕНИЕ ЗАЕМЩИКОВ ПО ЧИСЛУ ВЗЯТЫХ КРЕДИТОВ

credits['credit_count'].value_counts()


In [ ]:
plt.subplot(1,1,1)
sns.countplot(data = credits, x = 'credit_count')

Выводы:
- рекордсменом по количеству заемщиков является регион 31
- доля взявших два кредита примерно равна доле имеющих один кредит, но в целом поведение людей рациональное: количество кредитов обратно пропорционально числу заемщиков

In [ ]:
# ДОЛЯ НЕ ИМЕЮЩИХ КРЕДИТЫ

print(round(100*len(credits.query('credit_count == 0'))/credits.shape[0],2),'%')

In [ ]:
# РАСПРЕДЕЛЕНИЕ ЗАЕМЩИКОВ ПО ЧИСЛУ ПРОСРОЧЕННЫХ КРЕДИТОВ
# 2-ое условие - для исключения из подсчета лиц, не имеющих кредитов 

credits[(credits['overdue_credit_count'] > 0) & (credits['credit_count'] > 0)]['overdue_credit_count'].value_counts()

In [ ]:
plt.subplot(1,1,1)
sns.countplot(data = credits, x = 'overdue_credit_count', color = 'Green')

Вывод:
- и здесь поведение заемщиков ожидаемо рациональное - число заемщиков резко падает с ростом числа просроченных кредитов

In [ ]:
# ДОЛЯ ЛИЦ, ИМЕЮЩИХ ПРОСРОЧЕННЫЕ КРЕДИТЫ, ОТ ОБЩЕГО ЧИСЛА ЗАЕМЩИКОВ

print(round(100*len(credits.query('overdue_credit_count > 0'))/len(credits.query('credit_count > 0')),2),'%')

In [ ]:
# ОЦЕНКА ЗАКРЕДИТОВАННОСТИ

credit_count_gender = credits.groupby('gender')['credit_count'].sum()
print("Число выданных кредитов в зависимости от пола\n", credit_count_gender.sort_values())

credit_count_gender.plot.pie()
credit_count_gender.plot().legend(title = 'распределение кредитов по полу', bbox_to_anchor=(1, 1))

In [ ]:
credit_count_education = credits.groupby('education')['credit_count'].sum()
print("Число выданных кредитов в зависимости от образования\n", credit_count_education.sort_values())

credit_count_education.plot.pie()
credit_count_education.plot().legend(title = 'распределение кредитов по уровню образования', bbox_to_anchor=(1, 1))

In [ ]:
credit_count_living_region = credits.groupby('living_region')['credit_count'].sum()
print("Число выданных кредитов в зависимости от региона\n", credit_count_living_region.sort_values())

credit_count_living_region.plot.pie()
credit_count_living_region.plot().legend(title = 'распределение кредитов по регионам', bbox_to_anchor=(1, 1))

In [ ]:
credit_count_age = credits.groupby('age')['credit_count'].sum()
print("Число выданных кредитов в зависимости от возраста\n", credit_count_age.sort_values())

credit_count_age.plot.pie()
credit_count_age.plot().legend(title = 'распределение кредитов по возрастам', bbox_to_anchor=(1, 1))

In [ ]:
# представим разбиение по возрастам на линейной шкале

sns.barplot(data = credits, x = 'age', y = 'credit_count', estimator='size', color = 'Green', width = .4)

In [ ]:
credit_count_marital_status = credits.groupby('marital_status')['credit_count'].sum()
print("Число выданных кредитов в зависимости от семейного положения\n", credit_count_marital_status.sort_values())

credit_count_marital_status.plot.pie()
credit_count_age.plot().legend(title = 'распределение кредитов по семейному положению', bbox_to_anchor=(1, 1))

credits_by_marital_relative = round(100 * credits.groupby('marital_status').apply (lambda x: x['credit_count'].sum()/credits['credit_count'].sum()), 2)
print(f'\nотносительные доли взятых кредитов по семейному положению (в %):\n{credits_by_marital_relative}')

ВЫВОДЫ: охотнее всего берут кредиты
- люди в возрасте от 27 до 42 лет
- с уровнем образования 2 и 4
- в регионах с кодом 31, 73, 30, 23, 63
- больше всего кредитов берут заемщики с семейным статусом 3, меньше всего - со статусом 5 и 1, причем разница составляет почти 2500%
- разница в количестве кредитов между мужчинами и женщинами составляет чуть больше 10%

ВЫЯВЛЕНИЕ СВЯЗЕЙ МЕЖДУ ПРИЗНАКАМИ

In [ ]:
# ПОСТРОЕНИЕ ПАРНЫХ ГРАФИКОВ

columns_pairplot = credits.columns.drop(['client_id','okrug']) # будем считать okrug относительно незначимым для ускорения 
#columns_pairplot 
sns.pairplot(credits[columns_pairplot])

Ну, что тут сказать...
совершенно очевидно, например, что нет никакой связи между возрастом и регионом, возрастом и средней зарплатой (что логично), скоринговой оценкой и возрастом, скоринговой оценкой и регионом, количеством кредитов и регионом (что менее очевидно) и т.д.
В любом случае сильных связей не наблюдается.

Оценим силу связей в численном виде

In [ ]:
# МАТРИЦА КОРРЕЛЯЦИИ (ОЦЕНКА ВЕЛИЧИНЫ ЛИНЕЙНОЙ СВЯЗИ МЕЖДУ ПАРАМЕТРАМИ)

credits_corr = credits.corr().round(2)
credits_corr


ВЫВОДЫ:  самая сильная прямо пропорциональная связь существует
- между ежемесячным доходом и суммой взятых кредитов, 
- между ежемесячным доходом и средней зарплатой по региону
- между скоринговой оценкой и тарифом

Однако и эти корреляции не ярко выражены

In [ ]:
# СОХРАНИМ ПРИЗНАКИ СО СРАВНИТЕЛЬНО ЗНАЧИМЫМИ УРОВНЯМИ КОРРЕЛЯЦИИ (ВАРИАНТ 1)

selected_column = ['tariff_id','credit_sum','monthly_income','avregzarplata','credit_sum','score_shk']
credits_corr_strong_1 = credits[selected_column].corr().round(2)
credits_corr_strong_1

In [ ]:
# 'ТЕПЛОВАЯ КАРТА'

sns.heatmap(credits_corr_strong_1, annot=True, cmap='afmhot')

In [ ]:
# ПОПРОБУЕМ АВТОМАТИЗИРОВАТЬ ВЫБОР ПРИЗНАКОВ СО ЗНАЧИМЫМИ УРОВНЯМИ КОРРЕЛЯЦИИ (ВАРИАНТ 2)

# эта замена позволит отсечь диагональные значения на следующем шаге
credits_corr_replace = credits_corr.replace(1, np.NaN)

# отбор признаков, для которых значение модуля коэф. корреляции > 0.3
credits_corr_strong_2 = credits_corr_replace[credits_corr_replace.columns[abs(credits_corr_replace.max()) > 0.3]] 
credits_corr_strong_2

In [ ]:
sns.heatmap(credits_corr_strong_2, annot=True, cmap='afmhot')

Примеры более детальных диаграмм рассеивания

In [ ]:
# НА ПРИМЕРЕ ОТНОШЕНИЯ МЕЖДУ ЕЖЕМЕСЯЧНЫМ ДОХОДОМ И ЧИСЛОМ ВЗЯТЫХ КРЕДИТОВ

sns.regplot(data = credits, x ='monthly_income', y='credit_count')
credits['credit_count'].corr(credits['monthly_income'])

In [ ]:
# НА ПРИМЕРЕ ОТНОШЕНИЯ МЕЖДУ ЕЖЕМЕСЯЧНЫМ ДОХОДОМ И СУММОЙ ВЗЯТЫХ КРЕДИТОВ

sns.regplot(data = credits, x ='monthly_income', y='credit_sum')

In [ ]:
# ОЦЕНИМ ГРАФИЧЕСКИ СЛЕДУЮЩУЮ ГИПОТЕЗУ: ЧЕМ ВЫШЕ МЕСЯЧНЫЙ ЗАРАБОТОК, ТЕМ МЕНЬШЕ КОЛИЧЕСТВО ПРОСРОЧЕННЫХ КРЕДИТОВ

sns.regplot(data = credits, x ='monthly_income', y='overdue_credit_count')

ВЫВОДЫ: 1) связь между этими признаками слабая. 2) кредиты берут люди с разным уровнем достатка, но охотнее всего люди с невысоким достатком берут кредиты на небольшие суммы  3) график демонстрирует, что наибольшее количество просроченных кредитов сосредоточено в области низких заработков

In [ ]:
# ПОДТВЕРДИМ ПРЕДЫДУЩИЙ ВЫВОД, ВЫЧИСЛИВ КОЭФФИЦИЕНТ КОРРЕЛЯЦИИ

print(credits['credit_sum'].corr(credits['monthly_income']))
print(credits['monthly_income'].corr(credits['credit_sum']))

Примеры оценки нормального распределения

In [ ]:
# оценка нормального распределения (гистограмма) для суммы накопленных кредитов

sns.distplot(credits['credit_sum'])

# НЕ ИМЕЕТ НИЧЕГО ОБЩЕГО С НОРМАЛЬНЫМ РАСПРЕДЕЛЕНИЕМ,
# НИЖЕ СРАВНИМ С ДОВЕРИТЕЛЬНЫМ ИНТЕРВАЛОМ ДЛЯ СРЕДНЕГО

In [ ]:
# оценка нормального распределения (гистограмма) для скоринговой оценки

sns.distplot(credits['score_shk'])

# БЛИЗКО К НОРМАЛЬНОМУ РАСПРЕДЕЛЕНИЮ

In [ ]:
credits['overdue_credit_count'].corr(credits['monthly_income'])

In [ ]:
# ОЦЕНИМ ДОВЕРИТЕЛЬНЫЙ ИНТЕРВАЛ, НАПРИМЕР, ДЛЯ СРЕДНЕГО ЗНАЧЕНИЯ СУММЫ КРЕДИТА

credit_sum_mean = credits['credit_sum'].mean()
credit_sum_std = credits['credit_sum'].std()
print('среднее значение суммы кредита: ', credit_sum_mean)
print('стандартное отклонение для суммы кредита: ', credit_sum_std)
print(f'доверительный интервал для нормального распределения (среднее генеральной совокупности известно) с вероятностью 0.95: ',
        st.norm.interval(confidence = 0.95, loc=credit_sum_mean, scale = credit_sum_std))
print(f'доверительный интервал для t-распределения (среднее генеральной совокупности неизвестно) с вероятностью 0.95: ',
        st.t.interval(confidence = 0.95, df=credits.shape[0]-1, loc=credit_sum_mean, scale = credit_sum_std))
print(f'доверительный интервал для нормального распределения (среднее генеральной совокупности известно) с вероятностью 0.99: ',
        st.norm.interval(confidence = 0.99, loc=credit_sum_mean, scale = credit_sum_std))
print(f'доверительный интервал для t-распределения (среднее генеральной совокупности неизвестно) с вероятностью 0.99: ',
        st.t.interval(confidence = 0.99, df=credits.shape[0]-1, loc=credit_sum_mean, scale = credit_sum_std))        

Выводы: 
- чем выше вероятность, тем шире границы интервала
- отрицательную нижнюю границу в нашем случае, возможно, следует обнулить. но это не точно

<br>ОДНОФАКТОРНЫЙ ДИСПЕРСИОННЫЙ АНАЛИЗ (СРАВНЕНИЕ 2-Х ВЫБОРОК)

Проверка гипотезы №1<br>
чем выше средняя зарплата в регионе, тем меньше просроченных кредитов<br><br>

выборка 1: два региона с одинаковой средней зарплатой  - living_region = 17 & living_region = 57; avregzarplata = 27060;<br>
выборка 2: living_region = 30; avregzarplata = 66880<br>

размеры выборок близки

оценку произведем в соответствии с двухвыборочным t-критерием (поскольку о генеральных совокупностях данных нет), формирующем оценку гипотезы о равенстве средних значений - в нашем случае о равенстве средних значений выборок по просроченным кредитам 

- нулевая гипотеза: нет статистически значимых различий между средними для двух выборок
- первая гипотеза: существуют статистически значимые различия между средними для двух выборок

In [ ]:
region30 = credits[credits['living_region'] == 30]
region30 = region30.reset_index(drop=True)

print(f"записей по региону 30: {len(region30)}, средняя зарплата: {region30['avregzarplata'][0]}")


In [ ]:
# можно так
# reg17_57 = credits[(credits['living_region'] == 57) | (credits['living_region'] == 17)]

# но элегантнее
region17_57 = credits.query('living_region in [17,57]')
region17_57 = region17_57.reset_index(drop=True)


region17_57['avregzarplata'].value_counts()

print(f"всего записей по регионам 17 и 57: {len(region17_57)}, средняя зарплата: {region17_57['avregzarplata'][0]}")


In [ ]:
# УСЛОВИЕМ ПРАКТИЧЕСКОЙ ПРИМЕНИМОСТИ t-критерия ДЛЯ 2-Х НЕЗАВИСИМЫХ ВЫБОРОК ЯВЛЯЕТСЯ НОРМАЛЬНОСТЬ РАСПРЕДЕЛЕНИЯ ВЫБОРОК + РАВЕНСТВО ДИСПЕРСИЙ -> ОЦЕНИМ

print("std: ", region17_57['overdue_credit_count'].std())
sns.distplot(region17_57['overdue_credit_count'])

In [ ]:
print("std: ", region30['overdue_credit_count'].std())
sns.distplot(region30['overdue_credit_count'])

In [ ]:
# НИ НАМЕКА НА НОРМАЛЬНОСТЬ, НО ПОКАЖЕМ, КАК МОЖНО СДЕЛАТЬ ПОДОБНЫЙ ТЕСТ

In [ ]:
# здесь p-value - уровень значимости или вероятность, с которой мы можем утверждать, 
# что СРЕДНИЕ ЗНАЧЕНИЯ ЧИСЛА ПРОСРОЧЕННЫХ КРЕДИТОВ В 2-х ГЕНЕРАЛЬНЫХ СОВОКУПНОСТЯХ РАВНЫ


statistic, p_value = st.ttest_ind(region17_57['overdue_credit_count'], region30['overdue_credit_count'])
print(statistic, p_value)

In [ ]:
# В ДАННОМ СЛУЧАЕ НУЛЕВАЯ ГИПОТЕЗА СОСТОИТ В ТОМ, ЧТО СРЕДНИЕ ЗНАЧЕНИЯ ДВУХ ГЕНЕРАЛЬНЫХ СОВОКУПНОСТЕЙ РАВНЫ

if p_value>0.05:
    print('принимаем нулевую гипотезу с вероятностью', p_value)
else:
    print('отклоняем нулевую гипотезу, т.к. вероятность - ', p_value)

ВЫВОД: 
<p>существуют статистически значимые различия между выборками по количеству просроченных кредитов - средние по выборкам в сравниваемых регионах не равны<br>
а вот в регионах с высокой или низкой зарплатой количество просрочек - а Бог его знает. Посчитаем единственным реально доступным методом</p>

In [ ]:
print(region17_57['overdue_credit_count'].mean(), 
region30['overdue_credit_count'].mean())

Проверка гипотезы №2<br>
чем выше внутренняя скоринговая оценка, тем больше просроченных кредитов (на 1-ый взгляд между ними прямая зависимость)
<p>Сформируем две выборки по 200 строк с наивысшей и низшей скоринговой оценкой (за исключением 0, который соответствует клиентам без кредитов)</p>

In [ ]:
score_highest = credits.sort_values(by='score_shk', ascending = False)[:200]

In [ ]:
score_lowest = credits.query('score_shk != 0').sort_values(by='score_shk')[:200]

In [ ]:
statistic, p_value = st.ttest_ind(score_lowest['overdue_credit_count'], score_highest['overdue_credit_count'], nan_policy='omit')
print(statistic, p_value)
if p_value>0.05:
    print('принимаем нулевую гипотезу с вероятностью', p_value)
else:
    print('отклоняем нулевую гипотезу, т.к. вероятность - ', p_value)

Итак, существуют статистически значимые различия по просроченным кредитам между выборками для клиентов с высокой и низкой скоринговой оценкой</br>
в какой из них больше просроченных кредитов - да кто же знает... посчитаем доступным способом

In [ ]:
print(score_lowest['overdue_credit_count'].mean(), 
score_highest['overdue_credit_count'].mean())

В соответствии с поставленной аналитической задачей описания портрета клиента, который станет (и не станет) открывать кредитный счет можно сделать следующие частичные выводы:
<ul>кредиты берут люди с разным уровнем достатка, но охотнее всего люди с невысоким достатком берут кредиты на небольшие суммы </ul>
<ul>наибольшее количество просроченных кредитов сосредоточено в области низких заработков [хотя, как ни странно, тестовые данные показали, что в регионе с более высокой средней зарплатой просрочка по кредитам немного выше, чем в регионах с низким уровнем зарплат. Возможно, это связано с тем, величина кредитов в более "богатом" регионе выше]</ul>
<ul>рекордсменом по количеству заемщиков является регион 31</ul>
<ul>доля взявших два кредита примерно равна доле имеющих один кредит, в остальном число заемщиков резко падает с ростом числа взятых кредитов</ul>
<ul>при прочих равных можно ожидать, что чем выше ежемесячный доход, тем больше сумма взятых кредитов (по коэф-ту корреляции Пирсона)</ul>
<ul>наиболее склонны к пользованию кредитами:<br>
- люди в возрасте от 27 до 42 лет<br>
- с уровнем образования 2 и 4<br>
- в регионах с кодом 31, 73, 30, 23, 63<br>
- больше всего кредитов берут заемщики с семейным статусом 3, меньше всего - со статусом 5 и 1, причем разница составляет почти 2500%<br>
- разница между заемщиками мужчинами и женщинами в количестве кредитов невелика и составляет чуть больше 10%</ul>
<ul>вероятность просрочки выше у клиентов с высокой скоринговой оценкой</ul>